In [7]:
import pandas as pd
df_energia = pd.read_csv('../data/raw/spain_energy_market.csv')
df_energia.info()
display(df_energia.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40212 entries, 0 to 40211
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   datetime  40212 non-null  object 
 1   id        40212 non-null  int64  
 2   name      34734 non-null  object 
 3   geoid     10956 non-null  float64
 4   geoname   10956 non-null  object 
 5   value     40212 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.8+ MB


,datetime,id,name,geoid,geoname,value
0,2014-01-01 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,25.280833
1,2014-01-02 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,39.924167
2,2014-01-03 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,4.992083
3,2014-01-04 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,4.091667
4,2014-01-05 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,13.587500


In [10]:
# convierto data time de formato object a datetime de formato real 
df_energia['datetime'] = pd.to_datetime(df_energia['datetime'])
df_energia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40212 entries, 0 to 40211
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  40212 non-null  datetime64[ns]
 1   id        40212 non-null  int64         
 2   name      34734 non-null  object        
 3   geoid     10956 non-null  float64       
 4   geoname   10956 non-null  object        
 5   value     40212 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 1.8+ MB


In [ ]:
# vemos los conceptos de la columna
df_energia['name'].value_counts()

name
Precio mercado SPOT Diario ESP                                            1826
Precio mercado SPOT Diario FRA                                            1826
Precio mercado SPOT Diario POR                                            1826
Energía asignada en Mercado SPOT Diario España                            1826
Energía asignada en Mercado SPOT Diario Francia                           1826
Generación programada PBF Gas Natural Cogeneración                        1826
Generación programada PBF Solar fotovoltaica                              1826
Generación programada PBF UGH + no UGH                                    1826
Generación programada PBF Eólica                                          1825
Generación programada PBF total                                           1825
Demanda programada PBF total                                              1825
Demanda real                                                              1825
Generación programada PBF Nuclear              

In [ ]:
# identifico los paises que estan incluidos en el dataframe 
df_energia['geoname'].value_counts()

geoname
España      3652
Francia     3652
Portugal    3652
Name: count, dtype: int64

In [16]:
# vemos los conceptos que hay por cada pais 
print(df_energia.groupby('geoname')['name'].unique())

geoname
España      [Precio mercado SPOT Diario ESP, nan]
Francia     [Precio mercado SPOT Diario FRA, nan]
Portugal    [Precio mercado SPOT Diario POR, nan]
Name: name, dtype: object


In [17]:
# crear un dataset solo de precios de la energia 
concepto_precio = ['Precio mercado SPOT Diario ESP', 
    'Precio mercado SPOT Diario FRA', 
    'Precio mercado SPOT Diario POR']

df_precios = df_energia[df_energia['name'].isin(concepto_precio)].copy()

In [213]:
# creo una columna nueva para poder filtrar por pais evitando el nan que da al agrupar por geoname 
df_precios['pais'] = df_precios['name'].str.extract(r'(ESP|FRA|POR)')

In [20]:
df_precios['name'].value_counts()

name
Precio mercado SPOT Diario ESP    1826
Precio mercado SPOT Diario FRA    1826
Precio mercado SPOT Diario POR    1826
Name: count, dtype: int64

In [259]:
df_precios.head()

,datetime,id,name,geoid,geoname,value,pais
0,2014-01-01 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,25.280833,ESP
1,2014-01-02 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,39.924167,ESP
2,2014-01-03 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,4.992083,ESP
3,2014-01-04 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,4.091667,ESP
4,2014-01-05 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,13.587500,ESP


In [27]:
print(df_precios.groupby('pais')['name'].unique())

pais
ESP    [Precio mercado SPOT Diario ESP]
FRA    [Precio mercado SPOT Diario FRA]
POR    [Precio mercado SPOT Diario POR]
Name: name, dtype: object


In [284]:
# vemos datos al azar de 10 elementos 
print(df_precios['datetime'].sample(10))

4887   2017-05-20 22:00:00
540    2015-06-25 22:00:00
266    2014-09-24 22:00:00
819    2016-03-30 22:00:00
4366   2015-12-16 23:00:00
2343   2015-06-02 22:00:00
575    2015-07-30 22:00:00
3013   2017-04-02 22:00:00
4480   2016-04-08 22:00:00
4631   2016-09-06 22:00:00
Name: datetime, dtype: datetime64[ns]


In [217]:
# vemos los valores de los horarios del dataset
print(df_precios['datetime'].dt.hour.value_counts())

datetime
22    3213
23    2265
Name: count, dtype: int64


In [222]:
# pivotamos para hacer comparacios de datos creando un dataset 
df_pivot = df_precios.pivot(index='datetime', columns='geoname', values='value')
df_pivot.head()

geoname,España,Francia,Portugal
datetime,,,
2014-01-01 23:00:00,25.280833,28.71125,25.043750
2014-01-02 23:00:00,39.924167,28.39750,38.980417
2014-01-03 23:00:00,4.992083,12.99750,4.683750
2014-01-04 23:00:00,4.091667,23.08500,4.091667
2014-01-05 23:00:00,13.587500,31.29000,12.762500


In [ ]:
# quito el nombre geoname de la cabecera y reseteo el index
df_pivot.columns.name = None
df_pivot = df_pivot.reset_index()


In [232]:
# verificamos como quedo el dataset 
display(df_pivot.head())

,level_0,index,datetime,España,Francia,Portugal
0,0,0,2014-01-01 23:00:00,25.280833,28.71125,25.043750
1,1,1,2014-01-02 23:00:00,39.924167,28.39750,38.980417
2,2,2,2014-01-03 23:00:00,4.992083,12.99750,4.683750
3,3,3,2014-01-04 23:00:00,4.091667,23.08500,4.091667
4,4,4,2014-01-05 23:00:00,13.587500,31.29000,12.762500


In [ ]:
# verificamos una correlacion inicial 
df_pivot.drop(columns='datetime').corr()

,level_0,index,España,Francia,Portugal
level_0,1.000000,1.000000,0.408065,0.385574,0.418587
index,1.000000,1.000000,0.408065,0.385574,0.418587
España,0.408065,0.408065,1.000000,0.607336,0.996554
Francia,0.385574,0.385574,0.607336,1.000000,0.600304
Portugal,0.418587,0.418587,0.996554,0.600304,1.000000


In [ ]:
# seleccion de columnas que quiero observar y asegurando que solo eso muestre el dataset 
columnas_finales = ['datetime', 'España', 'Francia', 'Portugal']
df_pivot = df_pivot[columnas_finales]

In [242]:
# verificamos los cambios
df_pivot.drop(columns='datetime').corr()

,España,Francia,Portugal
España,1.000000,0.607336,0.996554
Francia,0.607336,1.000000,0.600304
Portugal,0.996554,0.600304,1.000000


In [244]:
# guardo el dataset pivot para usar y mostrar las graficas 
df_pivot.to_csv('../data/raw/precios_energia_comparativo.csv', index=False)